In [1]:
import os
import sys
sys.path.append('./src')
sys.path.append('./dependencies/indy-ros2/indy_description/urdf_files')
sys.path.append('../../../')
from utils import *
from gato_mpc import *
from gato_mpc_batch import *
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer
import meshcat.geometry as g
import meshcat.transformations as tf
import numpy as np
np.set_printoptions(linewidth=99999999)

# Load robot model
urdf_filename = "dependencies/indy-ros2/indy_description/urdf_files/indy7.urdf"
model, visual_model, collision_model = load_robot_model(urdf_filename)
model.gravity = pin.Motion.Zero()

# Create MPC solver and optimizer
#controller = MPC_GATO(model, N=32, dt=0.01)
controller = MPC_GATO_Batch(model, N=32, dt=0.01, batch_size=1)


In [2]:
# initial state
#xstart = np.zeros(controller.nx)
xstart = np.hstack((-0.10*np.ones(controller.nq), np.zeros(controller.nv)))

# goal ee positions
endpoints = np.array([
   np.array(controller.eepos(1.5*np.ones(controller.nq))),
   np.array(controller.eepos(np.zeros(controller.nq))), 
   np.array(controller.eepos(-0.9*np.ones(controller.nq)))
])

# Run MPC
q_trajectory, mpc_stats = controller.run_mpc(xstart, endpoints, 5)

Running MPC with 2500 steps, dt: 0.002
Endpoints:
[[ 1.35826737e-01 -6.80233138e-01 -2.84313361e-03]
 [ 1.51468905e-10 -1.86500000e-01  1.26750000e+00]
 [ 3.30300965e-01 -7.70697477e-01  5.35912989e-01]]
t:    0.000, goal dist:  1.34556
t:    0.050, goal dist:  1.34009
t:    0.100, goal dist:  1.30298
t:    0.200, goal dist:  1.05268
t:    0.350, goal dist:  0.38442
t:    0.400, goal dist:  0.24064
t:    0.550, goal dist:  0.39589
t:    0.700, goal dist:  0.59286
t:    0.800, goal dist:  0.66155
t:    0.950, goal dist:  0.68108
t:    1.100, goal dist:  0.64920
t:    1.150, goal dist:  0.63969
t:    1.350, goal dist:  0.61599
t:    1.400, goal dist:  0.61253
t:    1.600, goal dist:  0.60686
t:    1.650, goal dist:  0.60769
t:    1.900, goal dist:  0.61176
t:    2.200, goal dist:  0.60491
t:    2.300, goal dist:  0.60061
t:    2.450, goal dist:  0.59696
t:    2.550, goal dist:  0.59615
t:    2.700, goal dist:  0.59271
t:    2.800, goal dist:  0.59001
t:    2.950, goal dist:  0.58847
t:  

In [1]:
for key in mpc_stats:
    print(key)
    print(mpc_stats[key])

NameError: name 'mpc_stats' is not defined

In [3]:
stats = controller.solver.get_stats()

for key in stats:
    print(key)
    print(stats[key])
    if key == 'solve_time':
        print(np.mean(stats[key]['values']))
        print(np.max(stats[key]['values']))
        print(np.min(stats[key]['values']))
    

solve_time
{'values': [14997.0, 5850.0, 6397.0, 6652.0, 7740.0, 6592.0, 8273.0, 8305.0, 8839.0, 8235.0, 7387.0, 11946.0, 10437.0, 10494.0, 11480.0, 11760.0, 7560.0, 7841.0, 8925.0, 8286.0, 10274.0, 11263.0, 9831.0, 10323.0, 9137.0, 8746.0, 9382.0, 11892.0, 8887.0, 10795.0, 8604.0, 7304.0, 8353.0, 6822.0, 6860.0, 7185.0, 6639.0, 7227.0, 7895.0, 6358.0, 8540.0, 8155.0, 8205.0, 7942.0, 6860.0, 8812.0, 7046.0, 6847.0, 9512.0, 7452.0, 6224.0, 7088.0, 5951.0, 6624.0, 8648.0, 7307.0, 5165.0, 6278.0, 10452.0, 6014.0, 6258.0, 8184.0, 6359.0, 8348.0, 7476.0, 9654.0, 7118.0, 6401.0, 4923.0, 4935.0, 8994.0, 8125.0, 7871.0, 6757.0, 7798.0, 8565.0, 7083.0, 7244.0, 4733.0, 4544.0, 5351.0, 8249.0, 6841.0, 8247.0, 4848.0, 4011.0, 5546.0, 4054.0, 4540.0, 3996.0, 3754.0, 5146.0, 5439.0, 4219.0, 4418.0, 3625.0, 4571.0, 3737.0, 8808.0, 4353.0, 4247.0, 4225.0, 5463.0, 8746.0, 6314.0, 4429.0, 3596.0, 6565.0, 7618.0, 6147.0, 5720.0, 4569.0, 4669.0, 4524.0, 7550.0, 6538.0, 3542.0, 5256.0, 5241.0, 4400.0, 5265.

In [ ]:
# Setup pinocchio visualizer
viz = MeshcatVisualizer(model, collision_model, visual_model)
viz.initViewer()
viz.loadViewerModel()
# Visualize goals
for i, p in enumerate(endpoints):
    viz.viewer[f'point{i}'].set_object(
        g.Sphere(0.05), 
        g.MeshLambertMaterial(color=0xff0000))

    T = tf.translation_matrix(np.array(p[:]))
    viz.viewer[f'point{i}'].set_transform(T)
#print_stats(controller.solver.get_stats())
while 1:
    viz.play(q_trajectory=q_trajectory, dt=0.01)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [5]:
import pickle

print(q_trajectory)

# save trajectory to file
with open('q_trajectory.pkl', 'wb') as f:
    pickle.dump(q_trajectory, f)

[array([-0.10000215, -0.09995034, -0.10001428, -0.09973532, -0.10049627, -0.09970045]), array([-0.09999626, -0.0998874 , -0.10006218, -0.09946834, -0.1010091 , -0.09937014]), array([-0.09995853, -0.09966929, -0.10040356, -0.09892561, -0.10215607, -0.09848081]), array([-0.09993001, -0.09951316, -0.10071102, -0.09867053, -0.10275819, -0.09793127]), array([-0.10019403, -0.09893406, -0.10222653, -0.09829681, -0.10427466, -0.09615428]), array([-0.1005488 , -0.0984754 , -0.10358971, -0.09826648, -0.10516703, -0.09485713]), array([-0.10239259, -0.09691284, -0.10887264, -0.09938004, -0.10757959, -0.08959294]), array([-0.10401465, -0.09575326, -0.11306962, -0.10072823, -0.10908071, -0.08537751]), array([-0.10919641, -0.09250383, -0.12536881, -0.10528646, -0.11285667, -0.07426548]), array([-0.11285042, -0.09038437, -0.13363976, -0.10860579, -0.11509966, -0.06769667]), array([-0.12233224, -0.08490463, -0.1553781 , -0.11750124, -0.11957864, -0.05210192]), array([-0.12813764, -0.08150659, -0.168991